# imports

In [11]:
#manipulação de dados
import pandas as pd
import numpy  as np

#visualização
import matplotlib.pyplot as plt
import seaborn           as sns
from IPython.core.display import HTML


# Helper functions

In [12]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    sns.set()
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# 1. Load Data

In [14]:
df_raw = pd.read_csv('C:/Users/Lavin/Documents/desafios/desafio_indicium/data/raw/desafio_manutencao_preditiva_treino.csv')

In [15]:
df_raw.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
0,1,M14860,M,298.1,308.6,1551,42.8,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,No Failure
2,5,L47184,L,298.2,308.7,1408,40.0,9,No Failure
3,6,M14865,M,298.1,308.6,1425,41.9,11,No Failure
4,7,L47186,L,298.1,308.6,1558,42.4,14,No Failure


**dataset dimensions**

In [17]:
df_raw.shape

(6667, 9)

Temos um total de 6667 linhas com nove colunas no dataset de teste

## 1.1 Checking data types 

In [19]:
#copia para alterações não afetarem os dados originais
df1 = df_raw.copy()

In [21]:
df1.dtypes

udi                        int64
product_id                object
type                      object
air_temperature_k        float64
process_temperature_k    float64
rotational_speed_rpm       int64
torque_nm                float64
tool_wear_min              int64
failure_type              object
dtype: object

Não será necessário nenhum modificação no tipo dos dados, visto que as colunas já estão em seus devidos formatos

## 1.2 Checking NA

In [22]:
df1.isnull().sum()

udi                      0
product_id               0
type                     0
air_temperature_k        0
process_temperature_k    0
rotational_speed_rpm     0
torque_nm                0
tool_wear_min            0
failure_type             0
dtype: int64

o dataset não apresenta nenhum dados faltante, então a analise inicial dos dados já pode ser feita

# 2. Data description 

'udi': categorico

'product_id': categorico

'type':categotico

'air_temperature_k': numérico

'process_temperature_k':numérico

'rotational_speed_rpm':numérico

'torque_nm':numérico

'tool_wear_min':numérico

'failure_type':categorico


In [27]:
cat_attribures = df1[['udi','product_id','type','failure_type']]
num_attribures = df1[['air_temperature_k','process_temperature_k','rotational_speed_rpm','torque_nm','tool_wear_min']]

## 2.1 Numerical statistics

In [29]:
num_attribures.agg(['min','max','mean','median','std','skew','kurtosis']).T

,min,max,mean,median,std,skew,kurtosis
air_temperature_k,295.3,304.5,299.992515,300.0,1.994710,0.126800,-0.819938
process_temperature_k,305.7,313.8,309.992620,310.0,1.488101,0.016152,-0.508652
rotational_speed_rpm,1168.0,2886.0,1537.419529,1503.0,177.182908,1.952833,7.310365
torque_nm,3.8,76.6,40.058512,40.2,9.950804,0.011572,0.008249
tool_wear_min,0.0,251.0,108.098095,108.0,63.359915,0.019810,-1.157345


Todas as médias e medianas apresentaão valores próximos